In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tqdm     # 반복문 진행률

import tensorflow as tf
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel
import torch

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
tokenizer.eos_token_id

1

In [ ]:
import urllib.request
from torch.utils.data import DataLoader, Dataset
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",)

('ChatBotData.csv', <http.client.HTTPMessage at 0x7fe64af5f550>)

In [ ]:
df = pd.read_csv('ChatBotData.csv')
df.head(5)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
df = df[:1000]

In [ ]:
# df = pd.read_csv('train.csv')
# df.head()

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
batch_size = 32

In [ ]:
division = '<unused0>'

In [ ]:
def get_chat_data():
    for question, answer in zip(df.Q.to_list(), df.A.to_list()):
        # start of sentence
        bos_token = [tokenizer.bos_token_id]

        # end of sentence
        eos_token = [tokenizer.eos_token_id]

        sent = tokenizer.encode(question + division + answer)

        # 결과값을 나누어 얻음
        yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types = tf.int32)

In [ ]:
dataset = dataset.padded_batch(batch_size = batch_size, padded_shapes=(None,),
padding_values=tokenizer.pad_token_id)

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [ ]:
steps = len(df) // batch_size + 1
print(steps)

32


In [ ]:
model.to('cuda:0')
model.train()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.notebook.tqdm(dataset, total=steps):
        # tf.GradientTape() : 실행된 모든 연산은 테이프에 기록
        with tf.GradientTape() as tape:
            result = model(batch, label = batch)
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)
        
        grads = tape.gradient(batch_loss, model.trainable_variables)
        adam.apply_gradients(zip(grads, model.trainable_variables))

    model.save_pretrained('model.h5')
    print('[Epoch: {:>4}] cost = {:>9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/32 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
import transformers
transformers.__version__

'4.24.0'

In [ ]:
from transformers import TFTrainer, TFTrainingArguments
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,    # tensorflow_datasets training dataset
)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\trainer_tf.py:115: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


In [ ]:
trainer.train()

ValueError: The training dataset must have an asserted cardinality

In [ ]:
model.save_pretrained('model.h5')

NameError: name 'model' is not defined

In [ ]:
def return_answer_by_chatbot(user_text):
    sent = '' + user_text + ''
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
    input_ids = tf.convert_to_tensor([input_ids])
    output = model.generate(input_ids, max_length=96, do_sample=True, top_k=20)
    sentence = tokenizer.decode(output[0].numpy().tolist())
    chatbot_response = sentence.split('\n')[1]
    return chatbot_response
     

In [ ]:
return_answer_by_chatbot('커피 한 잔 할까')

'커피'